**Set environment**

In [1]:
source ../config_duke.sh

In [25]:
readlink -f ../config_duke.sh

/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks/config_duke.sh


In [4]:
ls ${FD_RES}/source

Input1            TFX2_CpdA            TFX3_GW870086        TFX4_RU486
Input1_20x        TFX2_Dex             TFX3_Hydrocortisone  TFX4_ZK216348
Input2            TFX2_DMSO            TFX3_Mapracorat      TFX5_AZD2906
Input2_20x        TFX2_GW870086        TFX3_RU486           TFX5_AZD9567
Input3            TFX2_Hydrocortisone  TFX3_ZK216348        TFX5_CORT108297
Input3_20x        TFX2_Mapracorat      TFX4_AZD2906         TFX5_CpdA
Input4            TFX2_RU486           TFX4_AZD9567         TFX5_Dex
Input4_20x        TFX2_ZK216348        TFX4_CORT108297      TFX5_DMSO
Input5            TFX3_AZD2906         TFX4_CpdA            TFX5_GW870086
Input5_20x        TFX3_AZD9567         TFX4_Dex             TFX5_Hydrocortisone
library_size.txt  TFX3_CORT108297      TFX4_DMSO            TFX5_Mapracorat
TFX2_AZD2906      TFX3_CpdA            TFX4_GW870086        TFX5_RU486
TFX2_AZD9567      TFX3_Dex             TFX4_Hydrocortisone  TFX5_ZK216348
TFX2_CORT108297   TFX3_DMSO            TFX4_Mapra

## Set up script to split chromosomes within a bed files

In [22]:
cat > splitBed.sh << 'EOF'
#############################################
### Usage ./splitBed.sh input.bed outdir
#############################################

### get directory from command line
FP_IN=$1
FD_OT=$2

### loop through the input bed file and 
### grep each chromosome to a output file
for chr in `cut -f 1 $FP_IN | sort | uniq`;
do
    echo $chr
    grep -w $chr $FP_IN > $FD_OT/$chr.bed
done
EOF

In [23]:
cat splitBed.sh

#############################################
### Usage ./splitBed.sh input.bed outdir
#############################################

### get directory from command line
FP_IN=$1
FD_OT=$2

### loop through the input bed file and 
### grep each chromosome to a output file
for chr in `cut -f 1 $FP_IN | sort | uniq`;
do
    echo $chr
    grep -w $chr $FP_IN > $FD_OT/$chr.bed
done


## Split Input 20x

In [7]:
ls -d ${FD_RES}/source/Input?_20x

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/source/Input1_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/source/Input2_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/source/Input3_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/source/Input4_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/source/Input5_20x


In [9]:
ls -d ${FD_LOG}
echo ${NODE}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log
all


In [24]:
### run script using sbatch
sbatch -p ${NODE} \
    --array=0-4 \
    --mem 8G \
    -o ${FD_LOG}/prep_splitbed_input_20x.%a.txt \
    <<'EOF'
#!/bin/bash
### set environment
source ../config_duke.sh

### set directories
FD_BEDS=($(ls -d ${FD_RES}/source/Input?_20x/))
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FP_BED=${FD_BED}/*.bedpe
FD_OUT=${FD_RES}/data/"$(basename -- $FD_BED)"

### Start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Input  directory:  " ${FD_BED}
echo "Input  file:       " ${FP_BED}
echo "Output directory:  " ${FD_OUT}
echo "Show the first few lines of the input file"
head ${FP_BED}

### create the output directory in not exist
mkdir -p ${FD_OUT}

### run script to split the bed file
chmod +x splitBed.sh
echo "Splitting the input file"
./splitBed.sh ${FP_BED} ${FD_OUT}
echo "Done!"

EOF

Submitted batch job 26142533


In [17]:
cat ${FD_LOG}/prep_splitbed_input_20x.0.txt

Slurm Array Index:  0
Input  directory:   /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/source/Input1_20x/
Input  file:        /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/source/Input1_20x//Input1.bedpe
Output directory:   /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x


In [18]:
ls ${FD_RES}/data

Input   Input2  Input4  TFX2_Dex   TFX3_Dex   TFX4_Dex   TFX5_Dex   TFX_Dex
Input1  Input3  Input5  TFX2_DMSO  TFX3_DMSO  TFX4_DMSO  TFX5_DMSO  TFX_DMSO


In [ ]:
### create the output directory in not exist
mkdir -p $FD_OUT
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Input  directory:  " ${FD_DAT}
echo "Input  file:       " ${FP_DAT}
echo "Output directory:  " ${FD_OUT}


In [ ]:
### set environment


### set log file directory
FD_LOG=/gpfs/fs1/data/reddylab/Kuei/out/CombEffect_STARR/log

### run script using sbatch
sbatch -pnew,all \
    --array=0-4 \
    --mem 8G \
    -o ${FD_LOG}/prep_splitbed_input_20x.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories
FD_ALIGN=/data/reddylab/gjohnson/whole_genome_STARRseq/wgss3/alignment_and_processing/alignments

### get data directory and set output directory
FD_DATS=($(ls -d ${FD_ALIGN}/Input*/))
FD_DAT=${FD_DATS[${SLURM_ARRAY_TASK_ID}]}
FD_OUT=/data/reddylab/Kuei/out/CombEffect_STARR/data/"$(basename -- $FD_DAT)"
FP_DAT=${FD_DAT}/*.bedpe

### create the output directory in not exist
mkdir -p $FD_OUT
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Input  directory:  " ${FD_DAT}
echo "Input  file:       " ${FP_DAT}
echo "Output directory:  " ${FD_OUT}

echo "Show the first few lines of the input file"
head ${FP_DAT}

### run script to split the bed file
chmod +x splitBed.sh
echo "Splitting the input file"
./splitBed.sh ${FP_DAT} ${FD_OUT}
echo "Done!"

EOF